## Books Data Engineering ETL

Spring 2024, by Oliver Seymour


#### My Data Sources:

Goodreads books and reviews: https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks

Audible books and reviews: https://www.kaggle.com/code/satyanarayanam/cleaning-audible-dataset/input

***add another here

# Extract

## Load data

## Explore data

### Exploring the Goodreads dataset

### Exploring the Audible dataset

### Exploring the ****third dataset

## Cleaning the data

### Cleaning the Goodreads dataset

### Cleaning the Audible dataset

### Cleaning the ****third dataset

## Integrate the data

# Transform

## Normalize the data

# Load

## Set up the database

## Load in the data

# Queries